In [ ]:
install.packages("tsDyn")
library(tsDyn)
library(readxl)

In [3]:
df <- read.csv("/content/gics.csv")
df$Date <- as.Date(df$Date)
summary(df)

      Date              Energy        Utilities           IT        
 Min.   :1-01-20    Min.   :193.9   Min.   :116.7   Min.   : 204.7  
 1st Qu.:8-07-20    1st Qu.:421.3   1st Qu.:175.7   1st Qu.: 379.4  
 Median :16-03-20   Median :506.1   Median :218.7   Median : 670.9  
 Mean   :16-03-13   Mean   :503.7   Mean   :238.1   Mean   :1144.7  
 3rd Qu.:23-10-20   3rd Qu.:580.3   3rd Qu.:303.7   3rd Qu.:1533.1  
 Max.   :31-12-20   Max.   :749.4   Max.   :418.7   Max.   :4711.3  
 NA's   :6          NA's   :6       NA's   :6       NA's   :6       
   Materials      Industrials     ConsumerDiscretionary ConsumerStaples
 Min.   :110.0   Min.   : 133.6   Min.   : 127.3        Min.   :203.4  
 1st Qu.:225.4   1st Qu.: 312.3   1st Qu.: 286.0        1st Qu.:287.9  
 Median :290.7   Median : 463.4   Median : 562.3        Median :491.1  
 Mean   :317.4   Mean   : 510.3   Mean   : 670.0        Mean   :480.3  
 3rd Qu.:374.2   3rd Qu.: 653.2   3rd Qu.: 956.8        3rd Qu.:620.1  
 Max.   :617.0  

In [4]:
df$sp500_logreturns <- c(NA, diff(log(df$sp500)))
df$wti_spot_logreturns <- c(NA, diff(log(df$wti_spot)))
df <- na.omit(df)
head(df)

,Date,Energy,Utilities,IT,Materials,Industrials,ConsumerDiscretionary,ConsumerStaples,healthcare,Financials,⋯,RealEstate,sp500,wti_spot,VIX,FedFunds,inflation_yoy,UNRATE,INDPRO,sp500_logreturns,wti_spot_logreturns
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,14-01-20,288.84,139.93,316.34,178.17,280.46,272.62,238.60,344.68,400.20,⋯,114.63,1184.52,46.79,12.43,2.25,0.02844874,5.3,95.8785,-0.001408861,0.05989023
3,21-01-20,287.36,139.53,307.49,175.74,275.18,266.17,237.98,340.69,396.96,⋯,115.50,1167.87,47.85,14.36,2.28,0.02844874,5.3,95.8785,-0.014156052,0.02240161
4,28-01-20,293.39,142.75,308.91,175.57,278.66,264.30,239.43,338.96,396.53,⋯,111.19,1171.36,48.56,13.24,2.27,0.02844874,5.3,95.8785,0.002983890,0.01472903
5,4-02-20,311.19,147.91,316.27,181.67,283.76,269.95,243.74,345.94,408.69,⋯,115.11,1203.03,46.97,11.21,2.43,0.03053026,5.4,96.5709,0.026677907,-0.03329105
6,11-02-20,319.34,147.25,315.58,182.87,285.35,269.58,241.74,347.25,410.18,⋯,117.55,1205.30,46.08,11.43,2.50,0.03053026,5.4,96.5709,0.001885124,-0.01913008
7,18-02-20,333.71,146.51,311.66,187.72,283.72,268.48,241.13,351.56,399.95,⋯,116.77,1201.59,47.82,11.18,2.50,0.03053026,5.4,96.5709,-0.003082819,0.03706495


In [5]:
DF <- na.omit(df[, c("sp500_logreturns", "wti_spot_logreturns", "VIX")])
nrow(DF)

[1] 1042

In [6]:
Y <- ts(DF[, c("sp500_logreturns", "wti_spot_logreturns")], frequency = 52)
VIX <- DF$VIX

In [8]:
VIX_vector <- as.numeric(na.omit(DF$VIX))

model <- TVAR(
  data = Y,
  lag = 1,
  nthresh = 1,
  thDelay = 1,
  trim = 0.15,
  mTh = 1,
  thVar = VIX_vector,
  model = "TAR",
  commonInter = FALSE)

print(model)
cat("Optimal Threshold (θ):", model$Thresh, "\n")

Best unique threshold 23.7 
Model TVAR with  1  thresholds

$Bdown
                                 Intercept sp500_logreturns -1
Equation sp500_logreturns     1.553495e-03         -0.06397763
Equation wti_spot_logreturns -1.490841e-05          0.23913129
                             wti_spot_logreturns -1
Equation sp500_logreturns               -0.01522454
Equation wti_spot_logreturns             0.19845406

$Bup
                                Intercept sp500_logreturns -1
Equation sp500_logreturns     0.002158799         -0.08265601
Equation wti_spot_logreturns -0.002704539          0.35742541
                             wti_spot_logreturns -1
Equation sp500_logreturns                0.01455001
Equation wti_spot_logreturns            -0.23843979


Threshold value[1] "23.7"
Optimal Threshold (θ): 
